## OBSERVATION_PERIOD

See [observation_period](https://ohdsi.github.io/CommonDataModel/cdm54.html#observation_period). This table records the start and end of periods where a patient has been under supervision of the health system. Each person in the PERSON table must have at least one period in the OBSERVATION_PERIOD table. The fields in the table are as follows:

```{mermaid}
erDiagram
    OMOP_OBSERVATION_PERIOD {
        integer observation_period_id
        integer person_id
        date observation_period_start_date
        date observation_period_end_date
        integer period_type_concept_id
    }
```

It is assumed that as long as the person has clinical records in the other tables, he/she is under observation. The absence of these records indicates that there is no observation. **Observation periods should not overlap with each other**.

The script that handles the transformation from the original files to OMOP format is [genomop_observation_period.py](../examples/genomop_observation_period.py). In this case we have to gather all those files that contain information about a patient's interaction with the health system. Not all files have this info, e.g. sociodemo does not include info about anything other than date of birth, gender and the id itself. These files do not need to be taken into account.

Once we have all the interactions, it is necessary to eliminate duplicates and visits contained one inside another, e.g. specific pharmacy interactions during a hospitalization of several days. This is done by the function `remove_overlap()` in the [general](../bps_to_omop/general.py) module.

After this, it is necessary to define the time between visits that we consider must have passed for the patient to have ceased to be under observation. By default we indicate 365 days, but this can be modified in the params file. The function `group_dates()` in the [general](../bps_to_omop/general.py) module then joins together every interaction that each patient had if the time passed between them is below a certain threshold, usually 365 days.



The parameters file will be [genomop_observation_period_params.yaml](../examples/genomop_observation_period_params.yaml). The parameters file must have the following structure:

```YAML
input_dir: preomop/03_omop_initial/
output_dir: preomop/04_omop_intermediate/OBSERVATION_PERIOD/
input_files:
  - 02_Patologias_BPS.parquet
  - 03_MPA.parquet
n_days: 365
transformations:
  - 02_Patologias_BPS.parquet: function_name
```

The neccesary params are:
- `input_dir` is the path from `data_dir` to the directory where input data is.
- `output_dir` is the path from `data_dir` to the directory where data will be saved to.
- `input_files` is the list of files to be used.
- `n_days` is the number of days between interactions to still be considered under observation.
- `transformations` is a dict containing transformations (values) that have to be applied to each file (keys).
  - transformations are performed by functions that operate on pyarrow tables. See [table_transformer.py](../external/bps_to_omop/bps_to_omop/table_transformer.py).